In [1]:
## IMPORTS
import random
from deap import base
from deap import creator
from deap import tools
import numpy as np
import timeit
start = timeit.default_timer()
from stage import fullPrint

In [2]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)


In [3]:
def generate_random_num_attr():
  original = np.array([0.3552483393772611, 0.7578300933136081, 0.09057686913234943, 1.033179934185795, 1.2749134898394006, 0.9592455936953672, 0.9775812152327962, 0.9583027148594099])
  noises = []
  sum_list = []

  noises.append(np.random.normal(0, 0.5))
  noises.append(np.random.normal(0, 1))
  noises.append(np.random.normal(0, 0.1))
  noises.append(np.random.normal(0, 0.01))
  noises.append(np.random.normal(0, 0.1))
  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.1))

  for (item1, item2) in zip(original, noises):
      sum_list.append(item1+item2)
  
  return sum_list

In [4]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", generate_random_num_attr)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [5]:
## change to our functinon
from stage import getStage
import numpy as np
import pandas as pd
from datetime import date
import yahoo_fin.stock_info as yf
import random
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = set(nasdaqList + nyseList)
def evalOneMax(individual):
    print(individual[0])
    transaction = yf.get_data('AAPL', start_date="1995-01-02",end_date="2022-01-09", index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    for symbol in random.sample(allList,1000):
        print(symbol)
        df = getStage(symbol,individual[0])
        inStage = False
        buyTwice = False
        if df.empty:
            continue
        for index, element in df.iterrows():
            index = index.strftime('%Y-%m-%d')
            if element.Stage == "Stage 2" or element.Stage == "Buy":
                transaction.loc[index]['holding'].append((symbol,element.close,0))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                if element.Stage == "Buy" and inStage:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                    buyTwice = True
                inStage = True
                continue
            if element.Stage == "Sell":
                transaction.loc[index]['holding'].append((symbol,element.close,-1))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,-1))
                inStage = False
                buyTwice = False
    total = 100
    prevIndex = ''
    first = True
    transactionCopy = transaction
    transactionCopy['total'] = 0
    for index, element in transactionCopy.iterrows():
        index = index.strftime('%Y-%m-%d')
        if len(element.holding) == 0:
            transactionCopy['total'][index] = total
            continue
        if first:
            first = False
            transactionCopy['total'][index] = total
        else:
            prevData = transactionCopy.loc[prevIndex].holding
            if len(prevData):
                total = 0
                removeList = []
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            if element.holding[j][2] == -1:
                                removeList.append(element.holding[j])
                            break
                    total += close*prevData[i][2]
                transactionCopy['total'][index] = total
                for delete in removeList:
                    if delete in element.holding:
                        element.holding.remove(delete)
            else:
                transactionCopy['total'][index] = transactionCopy.loc[prevIndex].total
        if len(element.holding):
            allocation = total/len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
        prevIndex = index
    stockHolding  = 0
    for i in transactionCopy.iterrows():
        stockHolding += len(i[1]['holding'])
    print(stockHolding/(transactionCopy.shape[0]))
    print(transactionCopy.iloc[-1]['total'])
    if stockHolding/(transactionCopy.shape[0])<8:
        return 0
    return transactionCopy.iloc[-1]['total'],

In [6]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [7]:
def main():


    pop = toolbox.population(n=512)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
        
    # Begin the evolution
    for g in range(9):
        print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values        
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        # Replace population
        pop[:] = offspring
    
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        
        
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))


In [8]:
main()

[0.19422160351861698, -2.5217052069698, 0.14520574809638984, 1.0508505921730165, 1.140762886931354, 0.8685769206745945, 0.995262473518448, 0.9060603928001626]


C:\Users\henry\AppData\Local\Temp/ipykernel_25848/4252288288.py:23: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  for symbol in random.sample(allList,2):


BIOTU
HOTH
0.0
100
[-0.11044724169471953, -0.08429165896311652, 0.15430326145729575, 1.0233433196624144, 1.4365056931437652, 0.9422377445768554, 1.0016510179832852, 1.0314026749363192]
BTI


C:\Users\henry\AppData\Local\Temp/ipykernel_25848/4252288288.py:23: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  for symbol in random.sample(allList,2):


TSQ
0.0
100
[0.10862160597615492, 2.3383672586349995, 0.18212410114114702, 1.049211416136469, 1.1994863566180463, 0.9490817945345248, 0.9490803462731883, 1.0529867644889621]


...

C:\Users\henry\AppData\Local\Temp/ipykernel_25848/4252288288.py:23: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  for symbol in random.sample(allList,2):


0.0
100
[-0.009642979715278555, -0.26277840926689666, 0.1529752886571219, 1.0349371505867369, 1.2251172370291277, 0.9168999534690045, 0.917511703073611, 0.9695996165532799]
BGCP


C:\Users\henry\AppData\Local\Temp/ipykernel_25848/4252288288.py:23: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  for symbol in random.sample(allList,2):


TRUP


KeyboardInterrupt: 